# Functional inference



In [1]:
## setup and import packages 

import os
import pandas as pd
import qiime2 as q2
import requests

from qiime2 import Visualization

data_dir = 'Data'
    
%matplotlib inline

In [2]:
def fetch_ipath(ids: list, img_output_path: str, verbose: bool = False):
    """Fetches a enriched pathways map from iPATH3 for given IDs."""
    url = 'https://pathways.embl.de/mapping.cgi'
    
    # remove colon from EC names
    if ':' in ids[0]:
        ids = [x.replace(':', '') for x in ids]
    
    if verbose:
        print(f'Fetching iPATH3 diagram for ids: {ids}')
    params = {
        'default_opacity': 0.6,
        'export_type': 'svg',
        'selection': '\n'.join(ids)
    }   
    response = requests.get(url=url, params=params)
    
    with open(img_output_path, 'wb') as img:
        img.write(response.content)

In [3]:
# path to the picrust2 conda environment
picrust_env = '/opt/conda/envs/picrust2/bin'

<a id='picrust'></a>
## 1. Functional inference

In [4]:
%%script env picrust_env="$picrust_env" data_dir="$data_dir" bash

# append the env location to PATH so that qiime
# can find all required executables
export PATH=$picrust_env:$PATH

$picrust_env/qiime picrust2 full-pipeline \
    --i-seq $data_dir/1-rep-seqs_bac.qza \
    --i-table $data_dir/1-feature-table_bac.qza \
    --output-dir $data_dir/picrust2_results \
    --p-placement-tool sepp \
    --p-threads 2 \
    --p-hsp-method pic \
    --p-max-nsti 2 

Saved FeatureTable[Frequency] to: Data/picrust2_results/ko_metagenome.qza
Saved FeatureTable[Frequency] to: Data/picrust2_results/ec_metagenome.qza
Saved FeatureTable[Frequency] to: Data/picrust2_results/pathway_abundance.qza


QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In the `picrust2_results` subdirectory, 3 artifacts as `FeatureTable[Frequency]` semantic type and hold abundance information for various functional features:

1. KO metagenome - abundances of [KEGG](https://www.kegg.jp) orthologs
2. EC metagenome - abundance of enzymes (you could look some of them up [here](https://www.brenda-enzymes.org)
3. MetaCyc pathways - abundance of [MetaCyc](https://metacyc.org) pathways

In [9]:
metadata = pd.read_csv(f'{data_dir}/0-metadata_bac.tsv', sep='\t', header=0, index_col=0)

In [10]:
## look at the metadata to check the available categories

! qiime metadata tabulate \
    --m-input-file $data_dir/0-metadata_bac.tsv \
    --o-visualization $data_dir/0-metadata_bac.qzv

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.
Saved Visualization to: Data/0-metadata_bac.qzv


In [11]:
Visualization.load(f'{data_dir}/0-metadata_bac.qzv')

<visualization: Visualization uuid: b0457538-92ce-4d3e-9f6b-70350b3de92b>

read in all three artifacts using QIIME 2 Python API - we can view them as DataFrames:

In [12]:
## read all three artifacts using QIIME 2 Python API - we can view them as DataFrames

ko = q2.Artifact.load(f'{data_dir}/picrust2_results/ko_metagenome.qza').view(pd.DataFrame)
ec = q2.Artifact.load(f'{data_dir}/picrust2_results/ec_metagenome.qza').view(pd.DataFrame)
pa = q2.Artifact.load(f'{data_dir}/picrust2_results/pathway_abundance.qza').view(pd.DataFrame)

 Look at the contents of each of resulted tables

In [13]:
ko.head()

,K00001,K00002,K00003,K00004,K00005,K00006,K00007,K00008,K00009,K00010,...,K19777,K19778,K19779,K19780,K19784,K19785,K19787,K19788,K19789,K19791
ERR1842970,15449.813969,971.486189,34528.116336,3535.764948,7600.251321,0.000000,28.862673,20364.046310,3617.470286,7721.839866,...,70.163140,60.683262,0.000000e+00,0.000000e+00,3932.992737,0.000000e+00,0.023977,6.958197e-04,1371.012062,0.176941
ERR1842971,24337.732003,2321.649421,33233.880392,2981.170227,30825.547308,0.000000,57.444589,22027.097231,3252.271406,8862.652184,...,80.010124,28.670483,1.010501e-43,1.019405e-43,2476.815498,5.420460e-130,0.000000,1.002508e-59,1548.033978,0.000042
ERR1842972,14342.961169,960.408474,28959.808177,2610.467378,8702.058235,0.000000,530.975065,18624.631287,4602.460834,8358.751898,...,45.435638,22.880090,2.698830e+00,1.855271e+00,2535.299760,0.000000e+00,0.000000,0.000000e+00,1616.010723,0.000159
ERR1842973,12017.160329,817.032836,26826.928342,1573.399263,5191.602392,0.005725,11.432426,10655.419661,1746.203962,4707.490882,...,6.763428,0.009178,9.207730e-49,9.288866e-49,2175.476313,1.010187e+01,0.005649,1.681534e-03,1354.902476,1.863283
ERR1842974,9622.984324,686.467483,20649.067462,1586.746095,8960.118044,0.000000,181.980799,15223.869999,3589.761646,6050.617238,...,36.840059,82.187635,0.000000e+00,0.000000e+00,2382.522254,4.130646e+00,0.001444,3.230048e-01,1583.781577,0.000000


In [14]:
ec.head()

,EC:1.1.1.1,EC:1.1.1.10,EC:1.1.1.100,EC:1.1.1.101,EC:1.1.1.102,EC:1.1.1.103,EC:1.1.1.105,EC:1.1.1.107,EC:1.1.1.108,EC:1.1.1.11,...,EC:6.4.1.8,EC:6.5.1.1,EC:6.5.1.2,EC:6.5.1.3,EC:6.5.1.4,EC:6.5.1.5,EC:6.5.1.6,EC:6.5.1.7,EC:6.6.1.1,EC:6.6.1.2
ERR1842970,44650.950511,92.439958,143848.117824,1.964992e-08,54.075241,3875.947110,1.224978,47.145824,1362.852930,28.862673,...,2.495300,9946.704888,61545.437557,16559.981016,883.844762,165.586706,3531.382305,3531.382305,27650.130099,35385.555374
ERR1842971,62720.523941,148.041974,152384.212303,2.636290e-27,142.583849,3813.824669,2.718365,104.896437,1012.921180,57.444589,...,0.149111,5546.880825,66940.525222,18681.739160,480.549778,65.059259,2286.052633,2286.052633,15668.870066,26005.100543
ERR1842972,40443.884877,113.025173,132862.647734,4.749872e-27,29.413927,2925.557471,7.403809,69.521268,891.150555,530.975065,...,0.188714,5668.948858,56537.924620,17769.282922,541.048698,13.775866,2413.540227,2413.540227,20936.738288,29057.587202
ERR1842973,29594.155475,99.688765,97811.595899,1.140156e-27,106.125050,2776.188614,6.083980,41.562785,948.102312,11.432426,...,25.862977,5646.766696,43973.364626,10091.179565,507.044075,14.532473,2132.786636,2132.786636,22608.261560,24874.074773
ERR1842974,32820.767588,18.680208,107682.192564,9.848864e-09,138.062521,3982.605918,0.042111,35.311147,1030.913782,181.980799,...,0.067717,5104.755972,45196.590257,15645.466503,699.376339,8.515516,1651.802933,1651.802933,3281.599633,16280.321819


In [15]:
pa.head()

,1CMET2-PWY,3-HYDROXYPHENYLACETATE-DEGRADATION-PWY,AEROBACTINSYN-PWY,ALL-CHORISMATE-PWY,ANAEROFRUCAT-PWY,ANAGLYCOLYSIS-PWY,ARG+POLYAMINE-SYN,ARGDEG-PWY,ARGORNPROST-PWY,ARGSYN-PWY,...,THISYN-PWY,THREOCAT-PWY,THRESYN-PWY,TRNA-CHARGING-PWY,TRPSYN-PWY,TYRFUMCAT-PWY,UBISYN-PWY,UDPNAGSYN-PWY,VALDEG-PWY,VALSYN-PWY
ERR1842970,52346.150689,934.156088,3.040647,3870.324345,42970.456693,63668.502233,21935.119260,226.036347,13264.712099,45083.314734,...,44468.142302,309.349553,62413.129261,61351.902333,47266.719352,4484.484209,8342.892306,40449.436070,143.307666,61824.060934
ERR1842971,56413.523340,831.370306,20.453622,5508.926323,46847.550535,70713.195125,16966.797283,198.384934,15751.126961,44624.327807,...,42225.970167,314.510945,60791.852851,64375.139723,45748.446330,5938.355727,8033.957521,38659.621301,235.454858,75229.578753
ERR1842972,47647.797484,687.881208,411.498246,5500.732680,37796.112232,58733.838610,18927.459528,297.455963,11679.139400,38928.271704,...,43327.432760,1863.888392,57195.214134,56394.345686,43184.453501,2307.532358,6687.211993,35324.842207,280.801577,53983.438560
ERR1842973,37309.921865,288.217116,0.604144,2008.952399,31496.961837,45345.076618,16486.027854,67.795221,10632.157195,32262.963282,...,32219.964975,244.301131,44779.196097,44065.441393,33630.676807,3431.862477,5238.464062,30786.188559,181.695532,41869.629507
ERR1842974,36878.822520,353.250362,0.292413,2975.230774,33060.071845,45947.564888,11309.931573,69.752082,10354.793078,25203.534608,...,32195.958904,634.156748,40899.531711,42279.108847,27464.629074,2812.640875,5056.826607,23718.959698,286.003034,38985.336678


You can see that they look just like the other feature tables we worked before with. The difference is that now they do not contain information about ASVs but about different levels of the functional profiles:

1. `ko` table: columns represent KEGG orthologs, as indicated by their names (e.g., **K**19777)
2. `ec` table: columns represent enzymes, as indicated by the Enzyme Commission numbers (e.g., **EC**:1.1.1.108)
3. `pa` table: columns represent entire pathways using the MetaCyc classification (e.g., ANAGLYCOLYSIS-PWY)

<a id='ipath'></a>
### 1.1 KEGG orthologs visualization

In [16]:
ko_meta = ko.merge(metadata[['Diets']], left_index=True, right_index=True)
ec_meta = ec.merge(metadata[['Diets']], left_index=True, right_index=True)
pa_meta = pa.merge(metadata[['Diets']], left_index=True, right_index=True)

calculating an average abundance of each KO, EC and pathway in each group

In [17]:
# collapse samples per diet types and calculate average abundance

ko_meta_avg = ko_meta.groupby('Diets').mean()
ec_meta_avg = ec_meta.groupby('Diets').mean()
pa_meta_avg = pa_meta.groupby('Diets').mean()

In [17]:
ko_meta_avg.head()

,K00001,K00002,K00003,K00004,K00005,K00006,K00007,K00008,K00009,K00010,...,K19777,K19778,K19779,K19780,K19784,K19785,K19787,K19788,K19789,K19791
Diets,,,,,,,,,,,,,,,,,,,,,
corn,13252.028012,884.135704,23651.045340,2260.277845,10733.953733,0.000000,173.282138,14716.065926,2722.162451,5499.498154,...,81.575545,68.853203,3.075969e-01,2.704955e-01,2344.612622,8.142615e-131,0.006193,0.015575,1291.257693,0.048167
grass,9545.315732,732.287206,21046.649428,1643.740295,7112.936005,0.037495,67.718219,11566.807662,1960.205124,4639.187286,...,16.463126,15.605550,2.484474e-48,2.506367e-48,2107.402170,5.208854e+00,0.001806,0.159915,1245.314649,0.721017
hay,12844.075673,772.263183,22553.639688,1572.156720,12386.494124,0.009775,19.146632,12652.716043,1876.444738,4246.313792,...,9.574571,2.527922,6.299361e-48,4.758332e-02,2328.318036,5.458807e+00,0.005698,0.088389,1364.356658,0.379204


In [18]:
# collapse samples per diet types and calculate average abundance
ko_meta = ko.merge(metadata[['Phase']], left_index=True, right_index=True)
ec_meta = ec.merge(metadata[['Phase']], left_index=True, right_index=True)
pa_meta = pa.merge(metadata[['Phase']], left_index=True, right_index=True)

In [19]:
ko_meta_avg = ko_meta.groupby('Phase').mean()
ec_meta_avg = ec_meta.groupby('Phase').mean()
pa_meta_avg = pa_meta.groupby('Phase').mean()

In [20]:
ko_meta_avg.head()

,K00001,K00002,K00003,K00004,K00005,K00006,K00007,K00008,K00009,K00010,...,K19777,K19778,K19779,K19780,K19784,K19785,K19787,K19788,K19789,K19791
Phase,,,,,,,,,,,,,,,,,,,,,
liquid phase,11652.186279,672.049955,21983.024282,1565.336909,9699.963787,0.006596,88.432031,11590.640936,2085.683969,4416.401787,...,30.949923,23.129073,7.726899e-03,1.119377e-01,2180.870825,4.620427,0.005266,0.066704,1280.682089,0.552473
rumen fluid,14392.178230,987.966302,26936.604600,2105.501660,12653.258650,0.001357,103.801933,15655.399135,2970.445427,6502.643506,...,33.276555,21.846664,2.998700e-01,2.061412e-01,2705.157147,3.989805,0.005628,0.078473,1502.000323,0.355185
solid phase,9597.054908,728.669836,18331.705573,1805.336291,7880.161425,0.039317,67.913025,11689.549560,1502.682917,3465.953939,...,43.386765,42.010938,5.743134e-48,5.793741e-48,1894.304857,2.057429,0.002804,0.118701,1118.246589,0.240730


In [21]:
# finding the most abundant features in each table
# according to the most abundant KOs, ECs and pathways 

def find_most_abundant(df: pd.DataFrame, frac):
    if 0 < frac < 1:
        frac = int(frac * len(df.columns))
    print(f'Saving {frac} most abundant features...')
    most_abundant = {
        smp: df.loc[smp, :].sort_values(ascending=False)[:frac]
        for smp in df.index
    }
    return most_abundant

ko_most_abundant = find_most_abundant(ko_meta_avg, 0.01)
ec_most_abundant = find_most_abundant(ec_meta_avg, 0.03)
pa_most_abundant = find_most_abundant(pa_meta_avg, 5)

Saving 105 most abundant features...
Saving 87 most abundant features...
Saving 5 most abundant features...


In [ ]:
print(f'5 most abundant KOs in the diets types: {ko_most_abundant["yes"].index[:5].tolist()}\n')

visualizing the KOs and ECs using- iPath 3. After fetching pathway maps for KOs and ECs per sample group and display them as SVGs. Then, producing interactive pathway map available and trying to interpreting different areas of the map.

In [65]:
for smp in ko_most_abundant.keys():
    fetch_ipath(ko_most_abundant[smp].index.tolist(), f'{data_dir}/kos_{smp}.svg')
    fetch_ipath(ec_most_abundant[smp].index.str.replace(':', '').tolist(), f'{data_dir}/ecs_{smp}.svg')

<a id='metacyc'></a>
### 1.2 Enriched pathways

The most abundant pathways in the sample groups. we used functions that defined to identify the four pathways that are most abundant in the samples.

In [34]:
print(f'4 most abundant pathways in the diets group are: {pa_most_abundant["yes"].index[:4].tolist()}\n')

KeyError: 'yes'

find out differentially abundant features between samples.

In [68]:
! qiime composition add-pseudocount \
    --i-table $data_dir/picrust2_results/pathway_abundance.qza \
    --o-composition-table $data_dir/picrust2_results/pathway_abundance_bac.qza

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.
Saved FeatureTable[Composition] to: Data/picrust2_results/pathway_abundance_bac.qza


In [73]:
! qiime composition ancom \
    --i-table $data_dir/picrust2_results/pathway_abundance_bac.qza \
    --m-metadata-file $data_dir/0-metadata_bac.tsv \
    --m-metadata-column Diets \
    --p-transform-function log \
    --o-visualization $data_dir/pa_ancom_abundance_bac.qzv

Saved Visualization to: Data/pa_ancom_abundance_bac.qzv


In [74]:
Visualization.load(f'{data_dir}/pa_ancom_abundance_bac.qzv')

<visualization: Visualization uuid: 969239db-a011-4725-8e36-5922393fb88e>

# Functional inference archaea

In [10]:
import os
import pandas as pd
import qiime2 as q2
import requests

from qiime2 import Visualization

data_dir = 'Data'
    
%matplotlib inline

In [11]:
def fetch_ipath(ids: list, img_output_path: str, verbose: bool = False):
    """Fetches a enriched pathways map from iPATH3 for given IDs."""
    url = 'https://pathways.embl.de/mapping.cgi'
    
    # remove colon from EC names
    if ':' in ids[0]:
        ids = [x.replace(':', '') for x in ids]
    
    if verbose:
        print(f'Fetching iPATH3 diagram for ids: {ids}')
    params = {
        'default_opacity': 0.6,
        'export_type': 'svg',
        'selection': '\n'.join(ids)
    }   
    response = requests.get(url=url, params=params)
    
    with open(img_output_path, 'wb') as img:
        img.write(response.content)

In [12]:
picrust_env = '/opt/conda/envs/picrust2/bin'

In [13]:
%%script env picrust_env="$picrust_env" data_dir="$data_dir" bash

# append the env location to PATH so that qiime
# can find all required executables
export PATH=$picrust_env:$PATH

$picrust_env/qiime picrust2 full-pipeline \
    --i-seq $data_dir/1-rep-seqs_arc.qza \
    --i-table $data_dir/1-feature-table_arc.qza \
    --output-dir $data_dir/picrust2_results_arc1 \
    --p-placement-tool sepp \
    --p-threads 2 \
    --p-hsp-method pic \
    --p-max-nsti 2 

Saved FeatureTable[Frequency] to: Data/picrust2_results_arc1/ko_metagenome.qza
Saved FeatureTable[Frequency] to: Data/picrust2_results_arc1/ec_metagenome.qza
Saved FeatureTable[Frequency] to: Data/picrust2_results_arc1/pathway_abundance.qza


QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In [21]:
metadata = pd.read_csv(f'{data_dir}/0-metadata_arc.tsv', sep='\t', header=0, index_col=0)

In [22]:
! qiime metadata tabulate \
    --m-input-file $data_dir/0-metadata_arc.tsv \
    --o-visualization $data_dir/0-metadata_arc.qzv

Saved Visualization to: Data/0-metadata_arc.qzv


In [23]:
Visualization.load(f'{data_dir}/0-metadata_arc.qzv')

<visualization: Visualization uuid: 835af58e-226c-4fb7-96d7-48abe8b106d6>

In [24]:
## read all three artifacts using QIIME 2 Python API and view as DaraFrame

ko = q2.Artifact.load(f'{data_dir}/picrust2_results_arc/ko_metagenome.qza').view(pd.DataFrame)
ec = q2.Artifact.load(f'{data_dir}/picrust2_results_arc/ec_metagenome.qza').view(pd.DataFrame)
pa = q2.Artifact.load(f'{data_dir}/picrust2_results_arc/pathway_abundance.qza').view(pd.DataFrame)

In [25]:
## kegg visualization
ko_meta = ko.merge(metadata[['Diets']], left_index=True, right_index=True)
ec_meta = ec.merge(metadata[['Diets']], left_index=True, right_index=True)
pa_meta = pa.merge(metadata[['Diets']], left_index=True, right_index=True)

In [26]:
# collapse samples per diet types and calculate average abundance

ko_meta_avg = ko_meta.groupby('Diets').mean()
ec_meta_avg = ec_meta.groupby('Diets').mean()
pa_meta_avg = pa_meta.groupby('Diets').mean()

In [27]:
# collapse samples per diet types and calculate average abundance
ko_meta = ko.merge(metadata[['Phase']], left_index=True, right_index=True)
ec_meta = ec.merge(metadata[['Phase']], left_index=True, right_index=True)
pa_meta = pa.merge(metadata[['Phase']], left_index=True, right_index=True)

In [28]:
ko_meta_avg = ko_meta.groupby('Phase').mean()
ec_meta_avg = ec_meta.groupby('Phase').mean()
pa_meta_avg = pa_meta.groupby('Phase').mean()

In [29]:
# finding the most abundant features in each table
# according to the most abundant KOs, ECs and pathways 

def find_most_abundant(df: pd.DataFrame, frac):
    if 0 < frac < 1:
        frac = int(frac * len(df.columns))
    print(f'Saving {frac} most abundant features...')
    most_abundant = {
        smp: df.loc[smp, :].sort_values(ascending=False)[:frac]
        for smp in df.index
    }
    return most_abundant

ko_most_abundant = find_most_abundant(ko_meta_avg, 0.01)
ec_most_abundant = find_most_abundant(ec_meta_avg, 0.03)
pa_most_abundant = find_most_abundant(pa_meta_avg, 5)

Saving 93 most abundant features...
Saving 82 most abundant features...
Saving 5 most abundant features...


In [30]:
## fetching pathways and visualization

for smp in ko_most_abundant.keys():
    fetch_ipath(ko_most_abundant[smp].index.tolist(), f'{data_dir}/kos_{smp}.svg')
    fetch_ipath(ec_most_abundant[smp].index.str.replace(':', '').tolist(), f'{data_dir}/ecs_{smp}.svg')

In [31]:
! qiime composition add-pseudocount \
    --i-table $data_dir/picrust2_results_arc/pathway_abundance.qza \
    --o-composition-table $data_dir/picrust2_results_arc/pathway_abundance_arc.qza

Saved FeatureTable[Composition] to: Data/picrust2_results_arc/pathway_abundance_arc.qza


In [32]:
! qiime composition ancom \
    --i-table $data_dir/picrust2_results_arc/pathway_abundance_arc.qza \
    --m-metadata-file $data_dir/0-metadata_arc.tsv \
    --m-metadata-column Diets \
    --p-transform-function log \
    --o-visualization $data_dir/pa_ancom_abundance_arc.qzv

Saved Visualization to: Data/pa_ancom_abundance_arc.qzv


In [33]:
Visualization.load(f'{data_dir}/pa_ancom_abundance_arc.qzv')

<visualization: Visualization uuid: 8dc6b31a-1e81-4830-9203-0bb1f65780a7>